In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 31.7 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.0 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 27.4 MB/s 
     |████████████████████████████████| 1.1 MB 61.6 MB/s 
     |████████████████████████████████| 140 kB 66.2 MB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 127 kB 60.5 MB/s 
     |████████████████████████████████| 94 kB 5.1 MB/s 
     |████████████████████████████████| 271 kB 74.9 MB/s 
     |█████████████

In [ ]:
# !rm -rf albert_results

In [1]:
import sys
sys.path.insert(0,'/content/')


In [3]:
import pandas as pd
from naive_bayes import NB
from BERT import BERT
from Albert import Albert
from xlnet import XLNET

from os import path
import glob


def initialize_df():
    if path.exists('accuracy.csv'):
        scores = pd.read_csv('accuracy.csv', index_col=[0])
    else:
        scores = pd.DataFrame({ 'Dataset': pd.Series(dtype='str'),
                                'NB': pd.Series(dtype='float'),
                                'Bert_1': pd.Series(dtype='float'),
                                'Bert_2': pd.Series(dtype='float'),
                                'Albert_1': pd.Series(dtype='float'),
                                'Albert_2': pd.Series(dtype='float'),
                                'XLNet_1': pd.Series(dtype='float'),
                                'XLNet_2': pd.Series(dtype='float'),
                                })

    if path.exists('training_time.csv'):
        time = pd.read_csv('training_time.csv', index_col=[0])
    else:
        time = pd.DataFrame({'Dataset': pd.Series(dtype='str'),
                                'NB': pd.Series(dtype='float'),
                                'Bert_1': pd.Series(dtype='float'),
                                'Bert_2': pd.Series(dtype='float'),
                                'Albert_1': pd.Series(dtype='float'),
                                'Albert_2': pd.Series(dtype='float'),
                                'XLNet_1': pd.Series(dtype='float'),
                                'XLNet_2': pd.Series(dtype='float'),
                                })
    return scores, time

if __name__ == "__main__":
    scores, times = initialize_df()

    datasets_dir = "datasets/*.csv"

    for filename in glob.glob(datasets_dir):
        print(filename[9:-4])
        score = []
        time = []

        score.append(filename[9:-4])
        time.append(filename[9:-4])

        # Naive Bayes
        model = NB(filename)
        a, t = model.pipeline()
        score.append(a)
        time.append(t)
        
        # BERT
        # model 7, 23, 13
        # skipped 23 since parameters were too similar. Dont mention skipping it in paper.
        parameters = [
            {'epochs': 5, 'warmup_steps': 0, 'learning_rate': 2e-5, 'adam_beta1': 0.9, 'adam_beta2': 0.999},
            {'epochs': 5, 'warmup_steps': 0, 'learning_rate': 1e-5, 'adam_beta1': 0.9, 'adam_beta2': 0.9},
        ]
        for p in parameters:
            model = BERT(filename, p)
            a, t = model.pipeline()
            score.append(a)
            time.append(t)
        
        # Albert
        # model 35, 24, 47
        parameters = [
            {'epochs': 5, 'warmup_steps': 1, 'learning_rate': 2e-5, 'adam_beta1': 0.8, 'adam_beta2': 0.9},
            {'epochs': 3, 'warmup_steps': 0, 'learning_rate': 1e-5, 'adam_beta1': 0.9, 'adam_beta2': 0.9},
        ]
        for p in parameters:
            model = Albert(filename, p)
            a, t = model.pipeline()
            score.append(a)
            time.append(t)

        # XLNet
        # model 24, 18, 2
        parameters = [
            {'epochs': 3, 'warmup_steps': 1, 'learning_rate': 1e-5, 'adam_beta1': 0.8, 'adam_beta2': 0.9},
            {'epochs': 3, 'warmup_steps': 1, 'learning_rate': 2e-5, 'adam_beta1': 0.8, 'adam_beta2': 0.999},
        ]
        for p in parameters:
            model = XLNET(filename, p)
            a, t = model.pipeline()
            score.append(a)
            time.append(t)

        score_df = pd.DataFrame([score], columns = ["Dataset", "NB", "Bert_1", "Bert_2", "Albert_1", "Albert_2","XLNet_1", "XLNet_2"])
        time_df = pd.DataFrame([time], columns = ["Dataset", "NB", "Bert_1", "Bert_2", "Albert_1", "Albert_2","XLNet_1", "XLNet_2"])
        scores = scores.append(score_df, ignore_index = True)
        times = times.append(time_df, ignore_index = True)

    scores.to_csv('accuracy.csv', index=False)
    times.to_csv('training_time.csv', index=False)

SMS_2
NB done


/content/naive_bayes.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace('\d+', '')
/content/BERT.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace('\d+', '')


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

  0%|          | 0/3 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
    

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the test set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: __index_level_0__, content. If __index_level_0__, content are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2800
  Batch size = 128


xlnet done
